### Raw Data Download Notebook

*Important Note:* This notebook runs using a different environment than this project's computational notebook. ObsPy is required, along with numpy and matplotlib

~~Zali et al. 2024 pulls horizontal seismic data from a single horizontal component from a single station, demeaned and detrended but not converted from instrument response. The data is pulled from March 12 to June 24, ~104 days, beginning 7 days before the Geldingadalir eruption began. The data is also downsampled to the minimum sampling rate necessary to observe the local volcanic tremor.~~

------------------

~~This notebook will download 100 days of data, beginning May 16 and ending August 24, from sensor AV.GSTD of the Alaska Volcano Observatory's network, located on the Great Sitkin Volcano. The Great Sitkin had an explosive eruption at 05:04 (UTC) May 26, 2021, followed by lava flow in mid-July https://avo.alaska.edu/eruption/great-sitkin-2021-05.~~

https://www.usgs.gov/observatories/hvo/science/eruption-kilauea-middle-east-rift-zone

In [1]:
import obspy
from obspy import UTCDateTime as utc
from obspy.clients.fdsn import Client

import numpy as np
import os

In [2]:
def process(st, buffer, freq, max_target_frequency): #freq is the original sampling frequency
    st.merge(fill_value='interpolate')
    tr = st[0].copy()

### Data Description

The seismic data is downloaded from the IRIS/Earthscope Database using ObsPy. The data downloads as a trace, an array of data with attached metadata, which is then packaged into a stream, which can contain multiple traces. The daily seismic data records are then saved as mseed files, which preserve this data+metadata structure, but requires ObsPy or other specialized software to open.

In [5]:
client = Client('IRIS')

#creating variables to download data
starttime = utc('2024-08-01T00:00:00')
endtime = starttime + 60 * (60*60*24)

#also add a buffer to both ends to chop off once the data has been filtered
#and downsampled, kind of arbitrary length, 5% of a day (default ObsPy taper length)
buffer = 60*60*24*0.05 #seconds

net = 'HV'
sta = 'UWB'
loc = '*' #wildcard, generally don't care about location code
cha = 'HHE' #horizontal component, as used in Zali et al

#create folder for numpy streams to go into and initialize filepath
!mkdir data
!mkdir data/raw
filepath = os.getcwd() + '/data/raw/'

#create arrays to save dates
dates = np.array([])

print(f'Downloading data from {starttime} to {endtime}')

#download the data piecemeal, here by day
for day in range(100):
    
    tr_length = 24*60*60
    
    # if file already exists, then skip
    if os.path.exists(filepath+str(day+1)+'_hvuwb.mseed'):
        print(f'Data for day {day + 1} already exists, skipping')
        starttime += tr_length
    else:
        try:
            # actually downloading
            st = client.get_waveforms(network=net,
                                    station=sta,
                                    location=loc,
                                    channel=cha,
                                    starttime=starttime-buffer,
                                    endtime=starttime+buffer+tr_length)

            # instrument sampling rate (hz)
            freq = st[0].stats.sampling_rate

            # merge traces within stream, linearly interpolating any gaps
            st.merge(fill_value='interpolate')

            # generate filename, day number in front for convenience of reading in
            name = str(day+1)+'_hvuwb.mseed'

            # save data as mseed, standard for storing seismic data. Preserves metadata and time series info
            st.write(filepath+name, format='MSEED')

            # adding date
            dates = np.append(dates, starttime.date)

            print(f'Downloaded day {day + 1}')
        except Exception as e:
            print(f'Error downloading data for day {day + 1}: {e}')
        
    starttime += tr_length

#save dates list for future use
np.save(filepath+'date_list.csv', dates)

print(f'data download complete, saved to {filepath}')



mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/raw’: File exists
Data for day 1 already exists, skipping
Data for day 2 already exists, skipping
Data for day 3 already exists, skipping
Data for day 4 already exists, skipping
Data for day 5 already exists, skipping
Data for day 6 already exists, skipping
Data for day 7 already exists, skipping
Data for day 8 already exists, skipping
Data for day 9 already exists, skipping
Data for day 10 already exists, skipping
Downloaded day 11
Downloaded day 12
Downloaded day 13
Downloaded day 14
Downloaded day 15
Downloaded day 16
Downloaded day 17
Downloaded day 18
Downloaded day 19
Downloaded day 20
Downloaded day 21
Downloaded day 22
Downloaded day 23
Downloaded day 24
Downloaded day 25
Downloaded day 26
Downloaded day 27
Downloaded day 28
Downloaded day 29
Downloaded day 30
Downloaded day 31
Downloaded day 32
Downloaded day 33
Downloaded day 34
Downloaded day 35
Downloaded day 36
Downloaded day 37
Downloa

### Data Modalities and Formats

#### Data Modalities
The dataset consists of seismic data collected from a single horizontal component (BHN) of the AV.GSTD station located on the Great Sitkin Volcano. The data is recorded continuously over a period of 100 days, capturing the seismic activity before, during, and after the volcanic eruption.

#### Data Formats
1. **MSEED (Mini-SEED) Files**:
    - The seismic data is stored in Mini-SEED format, which is a compact binary format used for storing time series data. Each file contains a day's worth of seismic data, including metadata such as the sampling rate and station information.
    - Example file name: `1_sitkin.mseed`, `2_sitkin.mseed`, ..., `100_sitkin.mseed`.
2. **Numpy Arrays**:
    - The dates corresponding to each day's seismic data are stored in a numpy array and saved as a CSV file (`date_list.csv`). This array helps in mapping the MSEED files to their respective dates.
    - Example: `dates = np.array([datetime.date(2021, 5, 16), datetime.date(2021, 5, 17), ...])`.

#### Data Processing
- The raw seismic data is downloaded using the ObsPy library from the IRIS/Earthscope Database.
- The data is merged and interpolated to fill any gaps, ensuring a continuous time series.
- The processed data is saved in the MSEED format, preserving both the time series and metadata.

This structured approach ensures that the seismic data is well-organized and easily accessible for further analysis and processing.

In [ ]:
# display some data from the mseed file
print(f'Example data from {name}')
print(st[0].stats)
print()

# display the numpy array
print(f'Example data from {name}')
print(st[0].data)
print()